# Verification models comparison

### Load dependencies

In [1]:
import pandas as pd
from ttest import paired_statistical_tests
from functools import reduce


2023-09-13 22:13:35.326968: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 22:13:35.387287: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 22:13:35.388204: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 22:13:36.311271: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
df = pd.read_csv('results_verification/5_2.csv', index_col='idx')
df.head()

,clf,accuracy,f1,precision,recall,threshold,Number of features,N-gram sizes,Oversample
idx,,,,,,,,,
0,RUSBoostClassifier,0.721189,0.720809,0.720434,0.721189,0.5,5,2,False
1,RUSBoostClassifier,0.718638,0.716091,0.713726,0.718638,0.5,5,2,False
2,RUSBoostClassifier,0.713509,0.711256,0.709138,0.713509,0.5,5,2,False
3,RUSBoostClassifier,0.718536,0.716011,0.713666,0.718536,0.5,5,2,False
4,RUSBoostClassifier,0.767139,0.672613,0.674730,0.767139,0.5,5,2,False


In [ ]:
df.columns

## Rank Models
1. Group the results by the unique model characteristics
2. For each group, you can compute the mean accuracy
3. Sort the models based on their mean accuracy:

In [10]:
MAIN_METRIC = 'accuracy'
COLUMNS = ['clf',  'Number of features', 'N-gram sizes', 'Oversample']
ranked_models = df.groupby(COLUMNS)[MAIN_METRIC].agg(['mean', 'std', 'count']).sort_values(by='mean', ascending=False)
ranked_models

,,,,mean,std,count
clf,Number of features,N-gram sizes,Oversample,,,
BalancedRandomForestClassifier,5,2,False,0.789143,0.029191,90
BalancedBaggingClassifier,5,2,False,0.775331,0.026847,90
RUSBoostClassifier,5,2,False,0.761599,0.026809,90
BalancedRandomForestClassifier,5,2,NEARMISS + BorderlineSMOTE,0.639284,0.118503,90
BalancedBaggingClassifier,5,2,NEARMISS + BorderlineSMOTE,0.615624,0.098201,90
RUSBoostClassifier,5,2,NEARMISS + BorderlineSMOTE,0.610745,0.075382,180


In [ ]:
def filter_df_based_on_lista(df, list_of_columns, model):
    conditions = [(df[list_of_columns[i]] == model[i]) for i in range(len(list_of_columns))]
    combined_condition = reduce(lambda x, y: x & y, conditions)
    return df[combined_condition]

In [ ]:
df.columns

In [ ]:
def pairwise_models_comparisons(df, ranked_models, num_comparisons=None, columns=COLUMNS, metric=MAIN_METRIC):
    results = []
    top_model = ranked_models.index[0]
    conditions = [(df[columns[i]] == top_model[i]) for i in range(len(columns))]
    combined_condition = reduce(lambda x, y: x & y, conditions) 
    
    top_model_data = filter_df_based_on_lista(df, columns, top_model)
    # If num_comparisons is None, compare against all models
    if num_comparisons is None:
        num_comparisons = len(ranked_models)

    for idx in ranked_models.index[1:num_comparisons]:
        current_model_data = filter_df_based_on_lista(df, columns, idx)
        print(top_model_data.columns)
        print(current_model_data.columns)
        t_stat, p_value, test_name = paired_statistical_tests(top_model_data, current_model_data)
        results.append((idx, t_stat, p_value))

    return results


In [ ]:
pd.DataFrame(pairwise_models_comparisons(df, ranked_models), columns=['clf', 'statistic_value', 'p-value', 'test'])